In [1]:
import pandas as pd
import numpy as np
import scipy

In [3]:
from scipy import spatial
from sklearn import preprocessing

In [4]:
reps = pd.read_excel('data.xlsx', sheet_name='Reps Profiling')
reps.head()

,Unnamed: 0,Unnamed: 0.1,First Name,Last Name,District,Party,Room,Phone Number,Email Address,Link,Name,County,% Dem,% Rep,Weeks in Office,No. Enviro
0,0,0,James,Arciero,2nd Middlesex,Democrat,277,(617) 722-2012,James.Arciero@mahouse.gov,https://malegislature.gov/Legislators/Profile/...,James Arciero,Middlesex County,0.663227,0.281705,572,5
1,1,1,Brian,Ashe,2nd Hampden,Democrat,236,(617) 722-2430,Brian.Ashe@mahouse.gov,https://malegislature.gov/Legislators/Profile/...,Brian Ashe,Hampden County,0.550959,0.390720,572,1
2,2,2,Bruce,Ayers,1st Norfolk,Democrat,167,(617) 722-2230,Bruce.Ayers@mahouse.gov,https://malegislature.gov/Legislators/Profile/...,Bruce Ayers,Norfolk County,0.612467,0.333084,1092,14
3,3,3,Ruth,Balser,12th Middlesex,Democrat,167,(617) 722-2810,Ruth.Balser@mahouse.gov,https://malegislature.gov/Legislators/Profile/...,Ruth Balser,Middlesex County,0.663227,0.281705,1092,6
4,4,4,Christine,Barber,34th Middlesex,Democrat,254,(617) 722-2220,Christine.Barber@mahouse.gov,https://malegislature.gov/Legislators/Profile/...,Christine Barber,Middlesex County,0.663227,0.281705,288,0


In [5]:
bills = pd.read_excel('data.xlsx', sheet_name="Bill Filtering")
bills.head()

,Unnamed: 0,Name,Link,H2836,H2007
0,0,James Arciero,https://malegislature.gov/Legislators/Profile/...,1,0
1,1,Brian Ashe,https://malegislature.gov/Legislators/Profile/...,0,0
2,2,Bruce Ayers,https://malegislature.gov/Legislators/Profile/...,1,0
3,3,Ruth Balser,https://malegislature.gov/Legislators/Profile/...,1,0
4,4,Christine Barber,https://malegislature.gov/Legislators/Profile/...,1,0


# Logistic Regression using Rep features

In [6]:
import warnings
warnings.filterwarnings('ignore')

**Clean and label with H2836**

In [10]:
df = pd.read_excel('data.xlsx', sheet_name='Reps Profiling')
df.drop(['Unnamed: 0', 'First Name', 'Last Name', 'District', 'Room', 'Phone Number', 'Email Address', 'Link'], axis=1, inplace=True)
df.head()

,Unnamed: 0.1,Party,Name,County,% Dem,% Rep,Weeks in Office,No. Enviro
0,0,Democrat,James Arciero,Middlesex County,0.663227,0.281705,572,5
1,1,Democrat,Brian Ashe,Hampden County,0.550959,0.390720,572,1
2,2,Democrat,Bruce Ayers,Norfolk County,0.612467,0.333084,1092,14
3,3,Democrat,Ruth Balser,Middlesex County,0.663227,0.281705,1092,6
4,4,Democrat,Christine Barber,Middlesex County,0.663227,0.281705,288,0


**Turn categorical labels into numbers**

In [11]:
df['Party'].value_counts()

Democrat      127
Republican     31
Unenrolled      1
Name: Party, dtype: int64

In [12]:
df['County'].value_counts()

Middlesex County     37
Suffolk County       19
Essex County         18
Worcester County     18
Norfolk County       15
Bristol County       14
Plymouth County      12
Hampden County       11
Barnstable County     5
Berkshire County      4
Hampshire County      3
Franklin County       2
Dukes County          1
Name: County, dtype: int64

In [14]:
def cleanDataset(df):
    #Update wrong cell
    df['Party'][155] = 'Republican'
    
    #Implement the One-Hot Encoding
    le = preprocessing.LabelEncoder()
    num_df = df[["Party", 'County']].apply(le.fit_transform)
    print(num_df)
    enc = preprocessing.OneHotEncoder()
    enc.fit(num_df)
    
    onehotlabels = enc.transform(num_df).toarray()
    clean_df = pd.DataFrame(onehotlabels)
    clean_df[['% Dem', '% Rep', 'Weeks in Office', 'No. Enviro']] = df[['% Dem', '% Rep', 'Weeks in Office', 'No. Enviro']]
    clean_df['Label'] = bills['H2836']

    return clean_df

clean = cleanDataset(df)
clean

     Party  County
0        0       8
1        0       6
2        0       9
3        0       8
4        0       8
..     ...     ...
154      1       0
155      1       5
156      0       6
157      1       4
158      0      12

[159 rows x 2 columns]


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,% Dem,% Rep,Weeks in Office,No. Enviro,Label
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.663227,0.281705,572,5,1
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.550959,0.390720,572,1,0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.612467,0.333084,1092,14,1
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.663227,0.281705,1092,6,1
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.663227,0.281705,288,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.539938,0.405843,288,2,0
155,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.641509,0.271856,288,2,0
156,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.550959,0.390720,156,0,1
157,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.584960,0.359986,468,0,0


**Training**

In [15]:
from sklearn.linear_model import LogisticRegression

log = LogisticRegression()

In [16]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(clean, test_size=0.4, random_state=3)
y_labels_train = train["Label"]
X_class_train = train.drop("Label", axis=1)
y_labels_test = test["Label"]
X_class_test = test.drop("Label", axis=1)

In [17]:
train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,% Dem,% Rep,Weeks in Office,No. Enviro,Label
45,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.729229,0.216372,156,13,1
35,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.663227,0.281705,988,0,1
134,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.518633,0.425797,468,16,0
50,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.612467,0.333084,1508,4,1
76,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.663227,0.281705,1300,10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.539938,0.405843,156,0,0
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.663227,0.281705,572,5,1
131,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.662853,0.268302,80,0,1
152,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.550959,0.390720,1404,0,1


In [18]:
log.fit(X_class_train, y_labels_train)

LogisticRegression()

**Predictions**

In [ ]:
predicts = log.predict(X_class_test)

pred_prob = log.predict_proba(X_class_test)
ones_prob = [x[1] for x in pred_prob]

In [ ]:
pred_prob[:5]

In [ ]:
ones_prob[:5]

In [ ]:
results = pd.DataFrame({"log":predicts, "prob_one":ones_prob, "actual":y_labels_test})
results.head()

## Overall Accuracy

In [ ]:
from sklearn.model_selection import cross_val_score

y = clean["Label"]
X = clean.drop("Label", axis=1)

In [ ]:
new_log = LogisticRegression()

In [ ]:
cvs = cross_val_score(new_log, X, y, cv=5)
cvs

In [ ]:
np.mean(cvs)

In [ ]:
len_test = len(results.index)
len_train = len(X_class_train.index)
num_correct = len(results[results['log']==results['actual']].index)
num_correct/len_test

In [ ]:
clean.head()

# Putting into Excel with all Predictions

In [ ]:
log = LogisticRegression()
log.fit(X_class_train, y_labels_train)

In [ ]:
predicts = log.predict(X) # run on entire dataset to get all scores
pred_prob = log.predict_proba(X)
ones_prob = [x[1] for x in pred_prob]

In [ ]:
ones_prob[:5]

In [ ]:
results = pd.DataFrame({"log":predicts, "prob_one":ones_prob, "actual":y})
results

In [ ]:
cross_val_score(LogisticRegression(), X, y, cv=5)

In [ ]:
np.mean(cross_val_score(LogisticRegression(), X, y, cv=5))

In [ ]:
len_all = len(results.index)
num_correct = len(results[results['log']==results['actual']].index)
num_correct/len_all

In [ ]:
num_correct

In [ ]:
len_all

In [ ]:
results['prob_one']

In [ ]:
reps = pd.read_excel('data.xlsx', sheet_name='Reps Profiling')
reps['Score'] = pd.Series(results['prob_one'])
reps['Label'] = bills['H2836']
reps

In [ ]:
with pd.ExcelWriter('Final.xlsx') as writer:
    reps.to_excel(writer)

## Into Json

In [ ]:
import json

In [ ]:
by_name = pd.read_excel("Final.xlsx")
by_score = pd.read_excel("Final.xlsx")

In [ ]:
by_name.index = df['Name']
by_score.index = df['Score']


by_name.drop(['ID', 'Room', 'County','Weeks in Office'], axis=1, inplace=True)
by_score.drop(['ID', 'Room', 'County','Weeks in Office'], axis=1, inplace=True)

In [ ]:
def jprint(obj):
    text = json.dumps(obj, sort_keys=True, indent=4)
    print(text)

In [ ]:
name = by_name.to_dict(orient='index')
jprint(name)

In [ ]:
with open("byName.json", "w") as write_file:
    # dump takes in data, then place to put it
    json.dump(name, write_file)

In [ ]:
import json # py library
file = open('byName.json', 'r')
dic = json.load(file)